# Importing Libraries

In [2]:
import pandas as pd
import re


In [3]:
df = pd.read_excel("PAN Number Validation Dataset.xlsx")
print(df.head(10))
total_records = len(df)
print(total_records)
                   

    Pan_Numbers
0    VGLOD3180G
1    PHOXD7232L
2    MGEPH6532A
3    JJCHK4574O
4    XTQIJ2330L
5    HTJYM3835H
6    YQTAP6661X
7    hvofe5635y
8  hyuij7902r  
9    idsmt3429e
10000


In [4]:
print(df.dtypes)

print('Total records = ',len(df))


Pan_Numbers    object
dtype: object
Total records =  10000


In [5]:
df['Pan_Numbers'] = df['Pan_Numbers'].astype('string').str.strip().str.upper()
print(df.dtypes)
print(df.head(10))
                                             

Pan_Numbers    string[python]
dtype: object
  Pan_Numbers
0  VGLOD3180G
1  PHOXD7232L
2  MGEPH6532A
3  JJCHK4574O
4  XTQIJ2330L
5  HTJYM3835H
6  YQTAP6661X
7  HVOFE5635Y
8  HYUIJ7902R
9  IDSMT3429E


In [6]:
print(df[df['Pan_Numbers'] == ''])

     Pan_Numbers
5019            
5020            


In [7]:
print(df[df['Pan_Numbers'].isna()])

     Pan_Numbers
5022        <NA>
5027        <NA>
5033        <NA>
5043        <NA>
5057        <NA>
...          ...
9961        <NA>
9972        <NA>
9986        <NA>
9987        <NA>
9997        <NA>

[965 rows x 1 columns]


In [8]:
df = df.replace({'Pan_Numbers':''},pd.NA)
print(df[df['Pan_Numbers'] == ''])
print(df[df['Pan_Numbers'].isna()])
#missing_values = len(df2[df2['Pan_Numbers'].isna()])
#print(missing_values)

Empty DataFrame
Columns: [Pan_Numbers]
Index: []
     Pan_Numbers
5019        <NA>
5020        <NA>
5022        <NA>
5027        <NA>
5033        <NA>
...          ...
9961        <NA>
9972        <NA>
9986        <NA>
9987        <NA>
9997        <NA>

[967 rows x 1 columns]


In [9]:
df = df.replace({'Pan_Numbers':''},pd.NA).dropna(subset='Pan_Numbers')
print("Total records: ",len(df))

Total records:  9033


In [10]:
print('Unique values = ',df['Pan_Numbers'].nunique())

Unique values =  9025


In [11]:
df = df.drop_duplicates(subset='Pan_Numbers',keep='first')
print('total records = ',len(df))

total records =  9025


# Validation

In [12]:
def has_adjacent_repitition(pan):
    for i in range(len(pan)-1):
        if pan[i] == pan[i+1]:
            return True
    return False

# has_adjacent_repitition('ABCDD')


# List Compression:
# return any(pan[i] == pan[i+1] for i in range(len(pan)-1))

In [13]:
def is_sequential(pan):
    for i in range(len(pan)-1):
        if ord(pan[i+1]) - ord(pan[i])!=1:
            return False
    return True

# is_sequential('ABCDE')
# is_sequential('ABCXY')

# List Compression:
# return all(pan[i+1] - pan[i] != 1 for i in range(len(pan)-1))

In [14]:
def is_valid_pan(pan):
    if len(pan) != 10:
        return False

    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$',pan):
         return False

    if has_adjacent_repitition(pan):
         return False

    if is_sequential(pan):
        return False

    return True

df['Status'] = df['Pan_Numbers'].apply(lambda x:"Valid" if is_valid_pan(x) else "Invalid")
print(df.head(10))
    

  Pan_Numbers   Status
0  VGLOD3180G    Valid
1  PHOXD7232L    Valid
2  MGEPH6532A    Valid
3  JJCHK4574O  Invalid
4  XTQIJ2330L  Invalid
5  HTJYM3835H    Valid
6  YQTAP6661X  Invalid
7  HVOFE5635Y    Valid
8  HYUIJ7902R    Valid
9  IDSMT3429E    Valid


In [22]:
valid_count = (df['Status'] == 'Valid').sum()
invalid_count = (df['Status'] == 'Invalid').sum()
missing_count = total_records - (valid_count + invalid_count)

In [23]:
print('Total records = ',total_records)
print('Valid = ',valid_count)
print('Invalid = ',invalid_count)
print('Missing = ',total_records - (valid_count + invalid_count))


Total records =  10000
Valid =  3193
Invalid =  5832
Missing =  975


In [17]:
df.head()

,Pan_Numbers,Status
0,VGLOD3180G,Valid
1,PHOXD7232L,Valid
2,MGEPH6532A,Valid
3,JJCHK4574O,Invalid
4,XTQIJ2330L,Invalid


In [18]:
df.tail()

,Pan_Numbers,Status
9994,ZMVA03025O,Invalid
9995,TNGUY429!V,Invalid
9996,SMMN33673G,Invalid
9998,RLAI69795T,Invalid
9999,YWY2482,Invalid


In [26]:
df_summary = pd.DataFrame({"TOTAL PROCESSED RECORDS": [total_records],
                           "TOTAL VALID COUNT": [valid_count],
                           "TOTAL INVALID COUNT": [invalid_count],
                           "TOTAL MISSING COUNT": [missing_count]})

print(df_summary)
                           

   TOTAL PROCESSED RECORDS  TOTAL VALID COUNT  TOTAL INVALID COUNT  \
0                    10000               3193                 5832   

   TOTAL MISSING COUNT  
0                  975  


In [27]:
with pd.ExcelWriter("PAN VALIDATION RESULT.xlsx")as writer:
    df.to_excel(writer,sheet_name="PAN VAlidatons",index=False)
    df_summary.to_excel(writer,sheet_name="SUMMARY",index=False)